# ScanningAgent

Looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [15:13<00:00, 182.79s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

'Title: Dyson Outlet Deals at eBay: Up to 60% off + extra $5 off + free shipping\nDetails: A selection of refurbished Dyson vacuum cleaners, hair care tools, and fans are discounted by up to 60% in eBay\'s outlet section.\xa0 Plus, if you pay with Venmo, you can knock an extra $5 off your order with coupon code "YUSFSID47J9KN87M". Pictured is the refurb Dyson SV29 Outsize Extra Cordless Vacuum Cleaner, which is $235 after the Venmo promo code (it originally cost $600 new). They come with Allstate warranties with details on the individual product pages. Shop Now at eBay\nFeatures: \nURL: https://www.dealnews.com/Dyson-Outlet-Deals-at-eBay-Up-to-60-off-extra-5-off-free-shipping/21744009.html?iref=rss-c196'

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Refurb Samsung Galaxy Watch 4 44mm Smartwatch for $34 + free shipping
Details: It's an old model but a great price for a big-brand smartwatch (you'd pay $55 for a refurb at Amazon). A 1-year Allstate warranty applies. Buy Now at eBay
Features: Auto Workout Tracking advanced sleep / continuous SpO2 monitoring 44mm AMOLED display 164 feet water resistance Android OS M

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description="The Refurbished Unlocked Samsung Galaxy S22 5G phone comes with 128GB of internal storage, perfect for users who need ample space for apps, photos, and videos. Known for its stunning camera capabilities and smooth performance, this device runs efficiently on Android OS, ensuring quick access to all your apps. The phone is backed by a one-year Allstate warranty, providing you with peace of mind regarding its refurbished condition. It's an ideal choice for anyone looking to upgrade their mobile experience without breaking the bank.", price=169.0, url='https://www.dealnews.com/products/Samsung/Unlocked-Samsung-Galaxy-S22-5-G-128-GB-Phone/470195.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The Refurbished Samsung Galaxy Watch 4 features a 44mm AMOLED display with advanced health and fitness tracking capabilities. It boasts auto workout tracking, continuous SpO2 monitoring, and is water-resistant up to 164 feet. This smartwatch runs on Android OS, model SM-R870, and comes with a 1-year Allstate warranty, making it an excellent choice for those looking to integrate smart technology into their fitness routine.', price=34.0, url='https://www.dealnews.com/products/Samsung/Samsung-Galaxy-Watch-4-44-mm-Smartwatch/330023.html?iref=rss-c142'), Deal(product_description='The Refurbished Unlocked Samsung Galaxy S22 5G offers 128GB of storage, making it a great deal for a premium smartphone experience. This device features a high-quality camera, impressive battery life, and the latest 5G technology for fast connectivity. It also comes with a 1-year warranty, ensuring reliability. This phone is an excellent option for users looking for a 